In [56]:
import pandas as pd

df=pd.read_csv("https://raw.githubusercontent.com/Alex-Mak-MCW/SpotifyDataScienceProject/main/Data/input.csv")

In [57]:
# DATA PREPPROCESSING (Basic Info)

# print(df.shape) # size of 1000 (entries), 27 (features)

print(df.info())
# df

# int (6): 
# Popularity,  Duration, key, mode, duration_ms, time_signature 

# float64(9): mostly audio features
# danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo

# object(11):
# Artist, Track, Track ID, Release Date, Album Name, Lyrics, type, id, uri, track_href, analysis_url

# bool (1)
# Explicit

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Artist            1000 non-null   object 
 1   Track             1000 non-null   object 
 2   Track ID          1000 non-null   object 
 3   Popularity        1000 non-null   int64  
 4   Duration          1000 non-null   int64  
 5   Release Date      1000 non-null   object 
 6   Album Name        1000 non-null   object 
 7   Explicit          1000 non-null   bool   
 8   Lyrics            974 non-null    object 
 9   danceability      1000 non-null   float64
 10  energy            1000 non-null   float64
 11  key               1000 non-null   int64  
 12  loudness          1000 non-null   float64
 13  mode              1000 non-null   int64  
 14  speechiness       1000 non-null   float64
 15  acousticness      1000 non-null   float64
 16  instrumentalness  1000 non-null   float64
 

In [58]:
# check null and duplicate values

# identify null data
print("Number of null values in the dataset:\n{0}".format(df.isnull().sum())) 
# There are 26 missing value in lyrics--> will be handled

# identify duplciate data
print("\nNumber of duplicate data in the dataset:\n{0}".format(df.duplicated().sum())) # No duplicate values

Number of null values in the dataset:
Artist               0
Track                0
Track ID             0
Popularity           0
Duration             0
Release Date         0
Album Name           0
Explicit             0
Lyrics              26
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
type                 0
id                   0
uri                  0
track_href           0
analysis_url         0
duration_ms          0
time_signature       0
dtype: int64

Number of duplicate data in the dataset:
0


In [59]:
# 2. drop duration_ms (Duration already exist)


df.drop(columns=['duration_ms'], inplace=True)
# df.columns

In [60]:
# drop object columns: track_href, analysis_url, Track ID, id, uri, and type
df.drop(columns=['track_href', 'analysis_url', 'Track ID', 'id', 'uri', 'type'], inplace=True)

print(df.shape)

(1000, 20)


In [61]:
df.shape

(1000, 20)

In [62]:
# DATA PREPPROCESSING (Part 1: numbers)

# 1. binary encode boolean feature (explicit) 
# print(df['Explicit'].value_counts())
df['Explicit']=df['Explicit'].astype(int)
# print(df['Explicit'].value_counts()) # 636 False, 364 True


In [63]:
# Descriptive/ Summary statistics for continuous data (15 columns)
numeric_summary=df.describe().style.set_caption('Summary Statistics for Continuous Data').format(precision=2).background_gradient(cmap='Blues')
display(numeric_summary)

,Popularity,Duration,Explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00
mean,78.26,216184.53,0.36,0.66,0.67,5.22,-6.00,0.54,0.10,0.20,0.01,0.18,0.49,121.25,3.98
std,7.27,54485.61,0.48,0.14,0.16,3.60,2.14,0.50,0.09,0.23,0.07,0.13,0.22,27.39,0.19
min,46.00,60719.00,0.00,0.21,0.09,0.00,-17.66,0.00,0.02,0.00,0.00,0.02,0.03,56.83,3.00
25%,74.00,184845.25,0.00,0.57,0.57,2.00,-7.09,0.00,0.04,0.03,0.00,0.09,0.32,99.99,4.00
50%,79.00,210820.00,0.00,0.67,0.68,5.00,-5.69,1.00,0.06,0.11,0.00,0.12,0.47,120.00,4.00
75%,84.00,238172.00,1.00,0.77,0.78,8.00,-4.58,1.00,0.12,0.29,0.00,0.21,0.65,138.95,4.00
max,97.00,592920.00,1.00,0.97,0.99,11.00,-1.48,1.00,0.55,0.98,0.85,0.89,0.97,203.76,5.00


In [69]:
# DATA PREPPROCESSING (Part 2: TEXT)

# check the object columns
# Select columns of type 'object'
object_columns = df.select_dtypes(include=['object']) # 5 columns left
# Artist, Track, Release Date, Album Name, Lyrics


# Convert object column to datetime
df['Release Date'] = pd.to_datetime(df['Release Date'])
# print(df['Release Date']) # success

Object columns in the DataFrame:
           Artist                                              Track  \
0    Taylor Swift                                       Cruel Summer   
1    Taylor Swift                                              Lover   
2    Taylor Swift                                          Anti-Hero   
3    Taylor Swift                                           cardigan   
4    Taylor Swift  Is It Over Now? (Taylor's Version) (From The V...   
..            ...                                                ...   
995     Lil Nas X                                           J CHRIST   
996     Lil Nas X                                      Old Town Road   
997     Lil Nas X                                             Panini   
998     Lil Nas X                                            HOLIDAY   
999     Lil Nas X                                              Rodeo   

                  Album Name  \
0                      Lover   
1                      Lover   
2     

In [94]:
# DATA has 1000,20: 20=15(int/float)+4(object/string)+1(datetime)

import lyricsgenius
genius=lyricsgenius.Genius("jCaG2w1CCDLgiXm8JGamJxOdA1Z8eMEKYJhjecctVLM270hsAJkwDBReNzhbZ4Fg", retries=3)

# try to 26 lyrics
# find the songs with missing values
missing_index = df[df.isnull().any(axis=1)].index

# print("Indexes of rows with missing values:")
missing_index_list=list(missing_index)
# print(missing_index_list) # [75, 80, 147, 168, 214, 254, 395, 399, 448, 506, 516, 539, 615, 645, 648, 649, 664, 714, 761, 778, 805, 808, 811, 815, 817, 829]

# 
temp=0

# impute it manually with the website
for i in missing_index_list:
    try:
        # Attempt to search for the song
        song = genius.search_song(df.loc[i, 'Track'].split("(", 1)[0], df.loc[i, 'Artist'])
        if song:
            # IMPUTE IT 
            df.loc[i, 'Lyrics']=song.lyrics
            # temp+=1
        else:
            song2 = genius.search_song(df.loc[i, 'Track'].split()[0], df.loc[i, 'Artist'])
            if song2:
                # IMPUTE IT 
                df.loc[i, 'Lyrics']=song2.lyrics
                # temp+=1

    except Exception as e:
        print("Error occurred:", e)

Searching for "Sunflower - Spider-Man: Into the Spider-Verse" by Post Malone...
Error occurred: Expecting value: line 3 column 1 (char 2)
Searching for "Cinderella" by Travis Scott...
Specified song does not contain lyrics. Rejecting.
Searching for "Cinderella" by Travis Scott...
Specified song does not contain lyrics. Rejecting.


In [96]:
# find the ones that are really missing:

missing_index = df[df.isnull().any(axis=1)].index

# print("Indexes of rows with missing values:")
missing_index_list=list(missing_index) 
print(missing_index_list)

# manual imputation by changing artist name
# impute it manually with the website
for i in missing_index_list:
    temp_artist_list=["Post Malone & Swae Lee", 'Future, Metro Boomin & Travis Scott']
    for j in temp_artist_list:
        try:
            # Attempt to search for the song
            song = genius.search_song(df.loc[i, 'Track'].split()[0], j)
            if song:
                # IMPUTE IT 
                df.loc[i, 'Lyrics']=song.lyrics
                temp+=1

        except Exception as e:
            print("Error occurred:", e)

# print(df[df.isnull().any(axis=1)].index)

[80, 147]
Searching for "Sunflower" by Post Malone & Swae Lee...
Done.
Searching for "Sunflower" by Future, Metro Boomin & Travis Scott...
Done.
Searching for "Cinderella" by Post Malone & Swae Lee...
Done.
Searching for "Cinderella" by Future, Metro Boomin & Travis Scott...
Done.
Int64Index([], dtype='int64')


In [98]:
print("Number of null values in the dataset:\n{0}".format(df.isnull().sum())) 

Number of null values in the dataset:
Artist              0
Track               0
Popularity          0
Duration            0
Release Date        0
Album Name          0
Explicit            0
Lyrics              0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
dtype: int64
